# Input size : 5 x 5 x 5
- board size : 5 x 5 
- P1 unique pieces : 2 
- P2 unique pieces : 2
- Colour

# Output size 5 x 5 x 13
- board size : 5 x 5 
- 13 possibles moves, 1 for each directions in ["N", "NE", "E", "SE", "S", "SW", "W", "NW"], plus the dragon, crab and tiger differents moves

### Layer codes

In [1]:
codes, i = {}, 0 
for nSquares in range(1,2):
    for direction in ["N", "NE", "E", "SE", "S", "SW", "W", "NW"]:
        codes[(nSquares, direction)] = i
        i += 1
        
for direction in ["E", "W", "S"]:
    codes[(2, direction)] = i
    i += 1
    
# codes[(2, "E")] = 8
# codes[(2, "W")] = 9
# codes[(2, "S")] = 10
codes[("2E", "1S")] = 11
codes[("2W", "1S")] = 12

In [2]:
codes

{(1, 'N'): 0,
 (1, 'NE'): 1,
 (1, 'E'): 2,
 (1, 'SE'): 3,
 (1, 'S'): 4,
 (1, 'SW'): 5,
 (1, 'W'): 6,
 (1, 'NW'): 7,
 (2, 'E'): 8,
 (2, 'W'): 9,
 (2, 'S'): 10,
 ('2E', '1S'): 11,
 ('2W', '1S'): 12}

In [3]:
class NNet():
    
    def __init__(self):
        pass

In [4]:
import numpy as np

In [5]:
policy = np.zeros((5, 5, 13))

In [6]:
columns = {k:v for v,k in enumerate("abcde")}

In [7]:
d2d3policy = np.zeros((5, 5, 13))
d2d3policy[columns["d"], 3-1, codes[(1, 'N')]] = 1

In [8]:
c2d3policy = np.zeros((5, 5, 13))
c2d3policy[columns["d"], 3-1, codes[(1, 'NE')]] = 1

In [9]:
# 50/50 chane for each move
openingPolicy = (d2d3policy + c2d3policy) / 2

In [10]:
openingPolicy[3][2]

array([0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ])

openingPolicy[2][3][8] mean that on the column C, on the line 3, the move was (2, 'E'). so the piece to move is in the column C and in the line 4

# Functionnal NN

In [11]:
# needed to do a class in order to have multiple instances of the nnet ? 

In [12]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

In [13]:
# input_block 
input_block = layers.Input(shape=(5, 5, 5))    # s: batch_size x board_x x board_y

In [14]:
# convolutionnal_layer 
x = layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="linear")(input_block)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

In [15]:
# Residual_layer
for _ in range(40):
    x = layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="linear")(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

In [16]:
# policy_head
policy_head = layers.Conv2D(filters=2, kernel_size=(1, 1), padding="same", activation="linear")(x)
policy_head = layers.BatchNormalization()(policy_head)
policy_head = layers.LeakyReLU()(policy_head)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Dense(13, activation="softmax", name="policy_head")(policy_head)

In [17]:
# value_head
value_head = layers.Conv2D(filters=1, kernel_size=(1,1), padding="same", activation="linear")(x)
value_head = layers.BatchNormalization()(value_head)
value_head = layers.LeakyReLU()(value_head)
value_head = layers.Flatten()(value_head)
value_head = layers.Dense(256, activation="linear")(value_head)
value_head = layers.LeakyReLU()(value_head)
value_head = layers.BatchNormalization()(value_head)
value_head = layers.Dense(1, activation="tanh", name="value_head")(value_head)

In [18]:
model = Model(inputs=[input_block], outputs=[policy_head, value_head])
model.compile(loss=['categorical_crossentropy','mean_squared_error'], optimizer="Adam")

In [20]:
# input size (1,5,5,5)